In [1]:
import os
import cv2
import gc
import mediapipe as mp
import numpy as np
import csv
import re

In [ ]:
#PRIORITY_X = 0.675
#PRIORITY_Y = 0.376
##Chossing best face by coordinates ##
def choose_best_face(detections, w, h, px=0.675, py=0.376):
    """
    To Select da face closest to da priority point (px, py) in normalized coords.
    """
    best_det = None
    best_dist = float("inf")

    for det in detections:
        bbox = det.location_data.relative_bounding_box

        cx = (bbox.xmin + bbox.width / 2) * w
        cy = (bbox.ymin + bbox.height / 2) * h

        dx = cx - (px * w)
        dy = cy - (py * h)
        dist = dx * dx + dy * dy  

        if dist < best_dist:
            best_dist = dist
            best_det = det

    return best_det


In [ ]:
##TEsting Testing for single Video .... Might wana use this for single videos if it might favor different priority coordinate
extract_lip_frames(
    video_path="C:/MajorProject/cropped_videos/Sample1.mp4",
    out_dir="lip_frames",
    start_time=0,
    end_time=10,
    priority_x=0.7,   # x cord from top left
    priority_y=0.3,  # y cord from top left
    verbose=True
)

🎥 29.97 fps | 2485 frames | 82.92s
🎬 Extracted 138 lip frames


True

In [ ]:
## just incase keeping the first function ... ##DONT RUN!!
def extract_lip_frames(
    video_path,
    out_dir,
    start_time=0,
    end_time=None,
    priority_x=0.5,
    priority_y=0.65,
    verbose=True,
    max_miss=15  # number of frames to tolerate missing faces
):
    """
    Extract lip/mouth frames using MediaPipe (CPU), 
    prioritizing the face nearest to the specified region px,py.
    Inserts blank frames if face is missing for max_miss frames.
    """

    os.makedirs(out_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception(f"Cannot open video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps if fps > 0 else 0

    if verbose:
        print(f"🎥 Video: {fps:.2f} fps | {total_frames} frames | {duration:.2f}s")

    if end_time is None or end_time > duration:
        end_time = duration

    cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)

    frame_idx = 0
    detected = 0
    miss_count = 0
    last_crop = None  # store last valid crop

    face_det = mp.solutions.face_detection.FaceDetection(
        model_selection=1,
        min_detection_confidence=0.3
    )

    try:
        while True:
            current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
            if current_time > end_time:
                break

            ret, frame = cap.read()
            if not ret:
                break

            # Resize large frames for speed
            h, w = frame.shape[:2]
            if w > 960:
                scale = 960 / w
                frame = cv2.resize(frame, None, fx=scale, fy=scale)
                h, w = frame.shape[:2]

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_det.process(rgb)

            crop = None
            if results.detections:
                det = choose_best_face(
                    results.detections,
                    w, h,
                    px=priority_x,
                    py=priority_y
                )

                if det is not None:
                    bbox = det.location_data.relative_bounding_box

                    x1 = int(bbox.xmin * w)
                    y1 = int(bbox.ymin * h)
                    bw = int(bbox.width * w)
                    bh = int(bbox.height * h)

                    # Focus on lower face (mouth region)
                    y1 = y1 + int(bh * 0.4)
                    bh = int(bh * 0.6)

                    # Padding + clipping
                    pad = 10
                    x1 = max(x1 - pad, 0)
                    y1 = max(y1 - pad, 0)
                    x2 = min(x1 + bw + 2 * pad, w)
                    y2 = min(y1 + bh + 2 * pad, h)

                    crop = frame[y1:y2, x1:x2]
                    last_crop = crop
                    detected += 1
                    miss_count = 0
            else:
                # If no face detected, allow a few missing frames
                miss_count += 1
                if last_crop is not None and miss_count <= max_miss:
                    crop = last_crop.copy()
                elif last_crop is not None and miss_count > max_miss:
                    # Create a blank frame same size as last crop
                    h_crop, w_crop = last_crop.shape[:2]
                    crop = np.zeros((h_crop, w_crop, 3), dtype=np.uint8)

            if crop is not None and crop.size > 0:
                cv2.imwrite(
                    os.path.join(out_dir, f"{frame_idx:06d}.jpg"),
                    crop
                )

            frame_idx += 1

            # garbage collection every 200 frames
            if frame_idx % 200 == 0:
                gc.collect()

    finally:
        face_det.close()
        cap.release()
        gc.collect()

    if verbose:
        print(f"🎬 Extracted {detected}/{frame_idx} frames")
        if detected == 0:
            print("⚠️ WARNING: No face detected in any frame")

    return detected > 0

###No nuLL

In [8]:
###No null
def extract_lip_frames(
    video_path,
    out_dir,
    start_time=0,
    end_time=None,
    priority_x=0.5,
    priority_y=0.65,
    verbose=True,
    max_miss=15
):
    """
    Extract lip/mouth frames using MediaPipe.
    If face is missing, reuses the last valid crop
    until a new face is detected.
    """

    os.makedirs(out_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception(f"Cannot open video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps if fps > 0 else 0

    if verbose:
        print(f"🎥 Video: {fps:.2f} fps | {total_frames} frames | {duration:.2f}s")

    if end_time is None or end_time > duration:
        end_time = duration

    cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)

    frame_idx = 0
    detected = 0
    miss_count = 0
    last_crop = None

    face_det = mp.solutions.face_detection.FaceDetection(
        model_selection=1,
        min_detection_confidence=0.3
    )

    try:
        while True:
            current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
            if current_time > end_time:
                break

            ret, frame = cap.read()
            if not ret:
                break

            h, w = frame.shape[:2]
            if w > 960:
                scale = 960 / w
                frame = cv2.resize(frame, None, fx=scale, fy=scale)
                h, w = frame.shape[:2]

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_det.process(rgb)

            crop = None

            if results.detections:
                det = choose_best_face(
                    results.detections,
                    w, h,
                    px=priority_x,
                    py=priority_y
                )

                if det is not None:
                    bbox = det.location_data.relative_bounding_box

                    x1 = int(bbox.xmin * w)
                    y1 = int(bbox.ymin * h)
                    bw = int(bbox.width * w)
                    bh = int(bbox.height * h)

                    # Mouth-focused crop
                    y1 += int(bh * 0.4)
                    bh = int(bh * 0.6)

                    pad = 10
                    x1 = max(x1 - pad, 0)
                    y1 = max(y1 - pad, 0)
                    x2 = min(x1 + bw + 2 * pad, w)
                    y2 = min(y1 + bh + 2 * pad, h)

                    crop = frame[y1:y2, x1:x2]
                    last_crop = crop
                    detected += 1
                    miss_count = 0

            else:
                miss_count += 1
                if last_crop is not None and miss_count <= max_miss:
                    crop = last_crop

            if crop is not None and crop.size > 0:
                cv2.imwrite(
                    os.path.join(out_dir, f"{frame_idx:06d}.jpg"),
                    crop
                )

            frame_idx += 1

            if frame_idx % 200 == 0:
                gc.collect()

    finally:
        face_det.close()
        cap.release()
        gc.collect()

    if verbose:
        print(f"🎬 Extracted {detected}/{frame_idx} frames")
        if detected == 0:
            print("⚠️ WARNING: No face detected in any frame")

    return detected > 0


With Capp


In [ ]:
########################
TSV_PATH = "C:/MajorProject/19-Dec.tsv"
VIDEO_DIR = "C:/dataset-dec-17/videos"
OUTPUT_ROOT = "C:/MajorProject/lip_framesfin"
############################

VIDEO_EXT = ".mp4"

PRIORITY_X = 0.675
PRIORITY_Y = 0.376

MAX_TSV_ROWS = 10   # Sabai tsv read garna None hala



def extract_end_time_from_text(text):
    tokens = text.strip().split()
    for token in reversed(tokens):
        if re.match(r"^\d+(\.\d+)?$", token) or ":" in token:
            return parse_time_to_seconds(token)
    raise ValueError(f"No valid time found in: {text}")


def parse_time_to_seconds(t):
    t = str(t).strip()
    if ":" in t:
        parts = [float(p) for p in t.split(":")]
        if len(parts) == 2:
            return parts[0] * 60 + parts[1]
        if len(parts) == 3:
            return parts[0] * 3600 + parts[1] * 60 + parts[2]
    return float(t)


def batch_extract_from_custom_tsv():
    os.makedirs(OUTPUT_ROOT, exist_ok=True)

    with open(TSV_PATH, encoding="utf-8") as f:
        reader = csv.reader(f, delimiter="\t")

        for idx, row in enumerate(reader, start=1):

            if MAX_TSV_ROWS is not None and idx > MAX_TSV_ROWS:
                break

            if len(row) < 2:
                continue

            base_name = row[0].strip()
            text_block = row[1]

            try:
                end_time = extract_end_time_from_text(text_block)
            except Exception:
                continue

            video_path = os.path.join(VIDEO_DIR, base_name + VIDEO_EXT)
            if not os.path.exists(video_path):
                continue

            out_dir = os.path.join(OUTPUT_ROOT, base_name)

            extract_lip_frames(
                video_path=video_path,
                out_dir=out_dir,
                start_time=0,
                end_time=end_time,
                priority_x=PRIORITY_X,
                priority_y=PRIORITY_Y,
                verbose=True
            )


if __name__ == "__main__":
    batch_extract_from_custom_tsv()


🎥 Video: 29.97 fps | 1285 frames | 42.88s
🎬 Extracted 234/331 frames
🎥 Video: 29.97 fps | 2781 frames | 92.79s
🎬 Extracted 181/181 frames
🎥 Video: 29.97 fps | 1422 frames | 47.45s
🎬 Extracted 181/181 frames
🎥 Video: 29.97 fps | 5030 frames | 167.83s
🎬 Extracted 211/211 frames
🎥 Video: 29.97 fps | 1355 frames | 45.21s
🎬 Extracted 301/301 frames
🎥 Video: 29.97 fps | 2772 frames | 92.49s
🎬 Extracted 181/181 frames
🎥 Video: 29.97 fps | 3427 frames | 114.35s
🎬 Extracted 301/301 frames
🎥 Video: 29.97 fps | 1436 frames | 47.91s
🎬 Extracted 181/181 frames
🎥 Video: 29.97 fps | 1703 frames | 56.82s
🎬 Extracted 271/271 frames
🎥 Video: 29.97 fps | 2243 frames | 74.84s
🎬 Extracted 151/151 frames
